### SPARC Technical question 2B: Happiness 
Applicant name: Johanan Lee Mahendran



Before I started modelling, I had to understand the question. Clarifying the definitions took up most of my time and even then it seems unclear how much these things stand to scrutiny.
There is the "common sense" assumption that a person is generally happier if their neighbours are happy.

<b> What does it mean to be happy? </b>  
This is such a awkward question in my opinion. One person's happiness can be another person's nightmare. And is there really an absolute objective measure of happiness? 
I generally consider someone happy when they are "content with their current situation", but I somewhat regret making this definition. 
Their "current situation" is an unspecified in time and space. Nobody is happy at all times and in all places. 


<b>What does it mean to be an "acquaintance"?</b>  
The question sheet specified that an acquaintance is "someone who interacts frequently with a person".
But what does interact mean here? 
There are many ways two people could interact. They could speak directly to each, they could chat on-line, use sign language, send each other messages on carrier pigeons, nod to each other as they pass by, play a game of chess, share a blunt and so on.   
Some interactions only occur when you have three or more people together: playing an online team game, participating in a race, acting in a play together, playing together as a band, etc. 


How frequent is frequently?  
Is it daily? Weeky? An unspecified but short period of time? What about people that meet consistently but over long periods? 
I decided to keep the meaning of "recently" vague and instead have my model occur only at a "slice" of time. 
A problem with only considering a "slice of time" is that people in general aren't happy or unhappy all the time. Evaluating the happiness of people in an instant would not indicate if the person is satisfied with their life. 
To give an example, a depressed person might be cheered up by friend every now and then, but they face general unhappiness.

For all intents and purposes, when I say "Person A is acquainted with person B" mean "Person A communicates with person B on an almost daily basis". 

<b> What are the external factors? </b>  
This is such a hard question in my opinion. First of all, it's not clear what is an external factor and what is one happiness. 
A person may be misreable even if they have many happy acquaintances (e.g. this person may be bullied by all their acquaintances) or someone may be happy even if they are all alone (e.g. living their dream life in their beach house).
Due to these considerations, I consider three additional factors:
1) Whether the interaction between person A and person B is positive (i.e. if person A's happiness increases so does person B's), negative (i.e. if person A's happiness increases, then person B's happiness decreases) or neutral (person A's and person B's happiness do not influence each other.)  
2) Each person has an extroversion score. i.e. an innate characteristic of the person which dictates if they are happy depending on the number of acquaintences or not. 
3) A third factor that acts like a baseline. This can be thought of as all the external factors that's not 1 or 2, lumped together into a single metric.

<b> How do I model the people and interactions? </b>    
I model my community of $N$ people using an directed graph. The nodes are the people and if an edge exists between two people are acquainted. 
There will be no edges from the person to themself. The edges are directed since Person A migh have a positive interaction with Person B but Person B might have a negative one. But in all cases, I take communication to be a two-way process and so all edges have an antiparallel edge-- if person A interacts with person B, then person B interacts with person A.  

I would also give each person an extroversion score. This would be a floating point value where positive values indicate more extroversion and negative values indicate more introversion. 
In addition to that, each edge has an associated weight. This is to indicate whether the interaction is positive or negative.
And finally, the baseline happiness.  
  
Using this model, 

An issue with the common sense assumption is that there may be a catch-22 situation. To evaluate Person A's happiness I need to know Person B's happiness but to know Person B's happiness I need to know Person A's happiness. 
One way to resolve this is to start everyone 

Each person has the property that 



I create a form to collect some data about how people perceive their happiness and the happiness of their acquaintances. 


In [1]:

import numpy as np
import matplotlib.pyplot as plt

def random_uniform():
    return np.random.uniform()*2 - 1

class Person:
    def __init__(self, extroversion, base_line):
        self.extroversion = extroversion
        self.base_line = base_line
        self.acquaintences = {}
    
    def make_acquaintence(self, other, weight = None):
        #creates a directed edge from self person to other person with weight
        if weight is None:
            weight = random_uniform()
        self.acquaintences[other] = weight

    def happiness(self):
        happy = self.base_line
        for acq in self.acquaintences:
            weight = self.acquaintences[acq]
            happy += weight * self.extroversion
        return min(max(0,happy),1)

class Graph():
    def __init__(self, persons = []):
        self.persons = persons
        
    def incidence_graph(self):
        n = len(self.persons)
        graph = np.zeros((n,n))
        for i in range(n):
            person = self.persons[i]
            acquaintences = person.acquaintences
            for j in range(n):
                acq = self.persons[j]
                if acq in acquaintences:
                    graph[i,j] = person.acquaintences[acq]
        return graph    

    def combine_graphs(self, other):
        return Graph(self.persons + other.persons)


## How does the graph look like?  
I'll be assuming that the people are connected to each other in a "small-world graph".
That is to say, people tend to form cliques -- if person A is acquatinted with person B, and person A is acquainted with person C, then it is likely that person B is acquainted with person C.
Furthermore, not only do people form cliques but these cliques are connected by only a few individuals. 

In [12]:
t = [1,2,9,8,7]
t[1:]

[2, 9, 8, 7]

In [14]:


class Clique(Graph):
    def __init__(self, n):
        self.persons = []
        for _ in range(n):
            extroversion = random_uniform()
            base_line = random_uniform()
            person = Person(extroversion, base_line)
            for acquaintance in self.persons:
                acquaintance.make_acquaintence(person)
                person.make_acquaintence(acquaintance)
            self.persons.append(person)
    def combine_cliques(self, other):
        if isinstance(other, Clique): 
            person = np.random.choice(self.persons)
            acquaintance = np.random.choice(other.persons)
            person.make_acquaintence(acquaintance)
            acquaintance.make_acquaintence(person)
            return self.combine_graphs(other)


In [16]:

def create_community(cliques = [2,2]):
    graph = Clique(cliques[0])
    for n in cliques[1:]:
        graph = graph.combine_cliques(Clique(n))
    return graph

create_community().incidence_graph()

array([[ 0.        ,  0.3845262 ,  0.        , -0.77448958],
       [-0.93144466,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.84910579],
       [-0.01330328,  0.        , -0.3322819 ,  0.        ]])

Bibliography and References:  
J. H. Fowler, N. A. Christakis, (2008) Dynamic spread of happiness in a large social network: longitudinal analysis over 20 years in the Framingham Heart Study, Downloaded from http://www.bmj.com/ on 10 April 2021.
